# Practical 5: Variance reduction methods in Monte Carlo simulation

Computational Finance with Python

[Alet Roux](https://www.york.ac.uk/maths/staff/alet-roux/) ([Department
of Mathematics](https://maths.york.ac.uk), University of York)

Click on the following to open this file in Google Colab:

<figure>
<a
href="https://colab.research.google.com/github/aletroux/comp-finance-python/blob/main/practicals/05_Monte_Carlo_variance_reduction_prac.ipynb"><img
src="https://colab.research.google.com/assets/colab-badge.svg"
alt="Open In Colab" /></a>
<figcaption>Open In Colab</figcaption>
</figure>

The aim of this practical is to explore variance reduction methods for
Monte Carlo simulation.

Throughout this practical we will focus on a European call option with
strike $K=100$ in the Black-Scholes model with parameters $S_0=100$,
$r=0.05$, $\sigma=0.1$ and $T=1$. The theoretical price of this option
can be calculated by using the function in the following code cell. Run
it to see the result.

In [1]:
# Black-Scholes parameters
S0 = 100
r = 0.05
sigma = 0.1
T = 1

# Strike price
K = 100

# Number of Monte Carlo simulations
n = 1000

from scipy.stats import norm
import math

def BScallprice (S0, K, T, r, sigma):
    """Calculates the theoretical price at time 0 of a European call option with strike K and maturity 
    date T in the Black-Scholes model with parameters r and sigma."""

    diff = sigma*math.sqrt(T)
    dplus = (math.log(S0/K) + (r + 0.5*sigma**2)*T)/diff
    dminus = dplus - diff
    price = S0 * norm.cdf(dplus) - math.exp(-r*T) * K * norm.cdf(dminus)

    return price

BSprice = BScallprice (S0, K, T, r, sigma)
print ("The theoretical price of the call option is", round(BSprice,4))

# 1. Plain Monte Carlo

The plain Monte Carlo procedure for pricing a call option in the
Black-Scholes model reads as follows:

1.  For $k=1,\ldots,n$:

    1.  Generate $W^Q_k\sim N(0, T)$.
    2.  Set
        $S_k=S_0e^{\left(r - \frac{1}{2}\sigma^2\right)t + \sigma W^Q_k}$.
    3.  Set $X_k = e^{-rT}(S_k - K)^+$.

2.  Compute estimate $\hat{C} = \frac{1}{n}\sum_{k=1}^n X_k$.

3.  Compute standard error
    $\mathop{\text{SE}} = \sqrt{\frac{1}{n(n-1)}\left(\sum_{k=1}^n X_k^2-n\hat{C}^2\right)}.$

The code in the following cell performs a plain Monte Carlo simulation
for pricing a European call option in the Black-Scholes model. It then
compares the Monte Carlo estimate against the theoretical value.

Run the code cell to view the results. Then study the code carefully to
see how it works.

In [2]:
import numpy as np

def MCprice_plain (S0, K, T, r, sigma, n, seed = 35):
    """Calculates the plain Monte Carlo estimate for the price at time 0 of a
    European call option with strike K and maturity date T in the Black-Scholes
    model with parameters r and sigma.
    
    Returns:
        estimate: Monte Carlo estimate
        standard_error: Standard error of Monte Carlo estimate
    """

    rng = np.random.default_rng(seed)

    # simulate W_T
    W = rng.normal(0, math.sqrt(T), n)

    # then calculate stock price
    S = S0*np.exp((r - 0.5*sigma**2)*T + sigma*W)

    # calculate discounted payoff
    X = np.maximum(S - K,0)*math.exp(-r*T)

    estimate = X.mean()
    standard_error = math.sqrt((np.sum(np.square(X)) - n*estimate**2)/n/(n-1))
    
    return estimate, standard_error

estimate_plain, standard_error_plain = MCprice_plain(S0, K, T, r, sigma, n)
print("Plain Monte Carlo estimate:", estimate_plain)
print("Standard error of Monte Carlo estimate:", standard_error_plain)

Notice that the function `MCprice_plain` has a keyword argument `seed`
with default value 35. This seed provides the starting point for the
random number generator. It is advised to keep the seed the same in each
of the cases studied below, as it makes it easier to compare the
results. (The value 35 was chosen at random and can be changed.)

# 2. Antithetic sampling

The following procedure uses antithetic sampling to produce a Monte
Carlo estimate of the price of a call option:

1.  For $k=1,\ldots,n$:

    1.  Generate $W^Q_k\sim N(0, T)$.
    2.  Set
        $S_k=S_0e^{\left(r - \frac{1}{2}\sigma^2\right)t + \sigma W^Q_k}$
        and
        $S'_k=S_0e^{\left(r - \frac{1}{2}\sigma^2\right)t - \sigma W^Q_k}$.
    3.  Set $X_k = e^{-rT}(S_k - K)^+$ and $X'_k = e^{-rT}(S'_k - K)^+$.

2.  Compute estimate
    $\hat{C} = \frac{1}{n}\sum_{k=1}^n \tfrac{1}{2}(X_k+X'_k)$.

3.  Compute standard error
    $\mathop{\text{SE}} = \sqrt{\frac{1}{n(n-1)}\left(\sum_{k=1}^n \tfrac{1}{4}(X_k+X'_k)^2-n\hat{C}^2\right)}.$

<span class="theorem-title">**Exercise 1**</span> Use the following code
block to implement the Monte Carlo estimation with antithetic sampling
described above.

Run the code and compare the standard error with that of the plain Monte
Carlo estimate.

In [3]:
def MCprice_antithetic (S0, K, T, r, sigma, n, seed = 35):
    """Calculates a Monte Carlo estimate with antithetic sampling for the price
    at time 0 of a European call option with strike K and maturity date T in
    the Black-Scholes model with parameters r and sigma.
    
    Returns:
        estimate: Monte Carlo estimate
        standard_error: Standard error of Monte Carlo estimate
    """

    # insert code here
    # hint: copy/paste the code for the plain Monte Carlo method here, and then
    # adjust it for antithetic sampling.
    # modify the lines below to calculate the actual values
    estimate = None
    standard_error = None
    
    return estimate, standard_error

estimate_antithetic, standard_error_antithetic = MCprice_antithetic(S0, K, T, r, sigma, n)
print("Monte Carlo estimate with antithetic sampling:", estimate_antithetic)
print("Standard error of Monte Carlo estimate with antithetic sampling:", 
      standard_error_antithetic)

# 3. Control variates

We will now perform a Monte Carlo estimate of the price of the call
option, using the discounted stock price (with known mean $S_0$) as a
control variable, and the coefficient $\hat{b}^\ast$ discussed in the
lectures. The following procedure calculates $\hat{b}^\ast$ and performs
the estimation with the control variable $e^{-rT}S_T$:

1.  For $k=1,\ldots,n$:
    1.  Generate $W^Q_k\sim N(0, T)$.
    2.  Set
        $S_k=S_0e^{\left(r - \frac{1}{2}\sigma^2\right)t + \sigma W^Q_k}$.
    3.  Set $X_k = e^{-rT}(S_k - K)^+$ and $Y_k = e^{-rT}S_k$.
2.  Set

$$\hat{b}^\ast = \frac{\sum_{i=1}^n(X_i-\bar{X})(Y_i-\bar{Y})}{\sum_{i=1}^n(Y_i-\bar{Y})^2},$$
where $\bar{X} = \frac{1}{n}\sum_{i=1}^n X_i$ and
$\bar{Y} = \frac{1}{n}\sum_{i=1}^n Y_i$.

1.  For $k=1,\ldots,n$: set

$$H_k = e^{-rT}(S_k - K)^+  - \hat{b}^\ast(Y_k - S_0).$$

1.  Compute estimate $\hat{C} = \frac{1}{n}\sum_{k=1}^n H_k$.

2.  Compute standard error
    $\mathop{\text{SE}} = \sqrt{\frac{1}{n(n-1)}\left(\sum_{k=1}^n H_k^2-n\hat{C}^2\right)}.$

<span class="theorem-title">**Exercise 2**</span> Use the following code
block to implement the Monte Carlo estimation procedure with control
variate as described above.

Run the code and compare the standard error with that of the plain Monte
Carlo estimate and the estimate with antithetic sampling.

In [5]:
def MCprice_control (S0, K, T, r, sigma, n, seed = 35):
    """Calculates a Monte Carlo estimate with control variable being the discounted
    stock price, for the price at time 0 of a European call option with strike
    K and maturity date T in the Black-Scholes model with parameters r and
    sigma.
    
    Returns:
        estimate: Monte Carlo estimate
        standard_error: Standard error of Monte Carlo estimate
    """

    # insert code here
    # modify the lines below to calculate the actual estimate and standard error!
    estimate = None
    standard_error = None
    
    return estimate, standard_error
    
estimate_control, standard_error_control = MCprice_control(S0, K, T, r, sigma, n)
print("Monte Carlo estimate with control variable:", estimate_control)
print("Standard error of Monte Carlo estimate with control variable:", standard_error_control)

# 4. Comparing variance reduction methods

It is interesting to compare the complexity (number of calculations) of
the various variance reduction techniques. Run the following code cells
to do this.

In [7]:
%timeit MCprice_plain(S0, K, T, r, sigma, n)

In [8]:
%timeit MCprice_antithetic(S0, K, T, r, sigma, n)

In [9]:
%timeit MCprice_control(S0, K, T, r, sigma, n)

Notice that, while the versions with variance reduction techniques use
the same number of random samples as the plain method, they do take
longer to run because of the increased complexity. This can be
considered a fair price to pay for the reduction in variance.

# 5. Combining variance reduction methods

It is of course possible to combine variance reduction methods. The two
methods we considered above can be combined in a number of ways.
Applying antithetic sampling to both the target sample and the control
variate leads to the following procedure:

1.  For $k=1,\ldots,n$:

    1.  Generate $W^Q_k\sim N(0, T)$.

    2.  Antithetic sampling: set

$$\begin{aligned}S_k&=S_0e^{\left(r - \frac{1}{2}\sigma^2\right)t + \sigma W^Q_k}, & S'_k &=S_0e^{\left(r - \frac{1}{2}\sigma^2\right)t - \sigma W^Q_k}.\end{aligned}$$

1.  Target sample: set

$$\begin{aligned} X_k &= e^{-rT}(S_k - K)^+, & X'_k = e^{-rT}(S'_k - K)^+.\end{aligned}$$

1.  Control variate: set

$$\begin{aligned} Y_k &= e^{-rT}S_k, & Y'_k = e^{-rT}S'_k.\end{aligned}$$

1.  Control variate coefficient: set

$$\hat{b}^\ast = \frac{\sum_{i=1}^n\left(\frac{1}{2}\left(X_i+X'_i\right)-\bar{X}\right)\left(\frac{1}{2}\left(Y_i+Y'_i\right)-\bar{Y}\right)}{\sum_{i=1}^n\left(\frac{1}{2}\left(Y_i+Y'_i\right)-\bar{Y}\right)^2},$$
where
$$\begin{aligned} \bar{X} &= \frac{1}{n}\sum_{i=1}^n \tfrac{1}{2}\left(X_i+X'_i\right), & \bar{Y} &= \frac{1}{n}\sum_{i=1}^n \tfrac{1}{2}\left(Y_i+Y'_i\right).\end{aligned}$$

1.  For $k=1,\ldots,n$: set

$$H_k = \tfrac{1}{2}\left(X_k+X'_k\right)  - \hat{b}^\ast\left(\tfrac{1}{2}\left(Y_k+Y'_k\right) - S_0\right).$$

1.  Compute estimate $\hat{C} = \frac{1}{n}\sum_{k=1}^n H_k$.
2.  Compute standard error
    $\mathop{\text{SE}} = \sqrt{\frac{1}{n(n-1)}\left(\sum_{k=1}^n H_k^2-n\hat{C}^2\right)}.$

<span class="theorem-title">**Exercise 3**</span>  

1.  Examine the procedure carefully to see how it works. Then implement
    it in Python.

2.  Compare the complexity and the amount of variance reduction with the
    methods studied above. What do you observe?

In [10]:
#Insert code here